In [10]:
!nvidia-smi

Mon Aug  5 20:36:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000000:3F:00.0 Off |                    0 |
| N/A   27C    P0              42W / 300W |      7MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [11]:
import torch

if torch.cuda.is_available():
    print("Current GPU:", torch.cuda.get_device_name(torch.cuda.current_device()))


Current GPU: NVIDIA A100 80GB PCIe


In [4]:
#downloading the text files
import requests

def download_file(url, filename):
    response = requests.get(url)
    response.raise_for_status()  
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(response.text)

ben_url = 'https://raw.githubusercontent.com/BibleNLP/ebible/main/corpus/ben-ben2017.txt'
eng_url = 'https://raw.githubusercontent.com/BibleNLP/ebible/main/corpus/eng-enggnv.txt'


download_file(ben_url, 'ben-ben2017.txt')
download_file(eng_url, 'eng-enggnv.txt')

In [ ]:
#preparing english and bangla toknizer models 
import sentencepiece as spm

corpus_file = 'eng-enggnv.txt'

spm.SentencePieceTrainer.train(
    input=corpus_file,
    model_prefix='english_bpe',
    vocab_size=32000,
    model_type='bpe',
    character_coverage=0.9995,
    max_sentence_length=10000
)

print(f"Tokenizer trained and saved as {model_prefix}.model and {model_prefix}.vocab")


In [ ]:
corpus_file = 'ben-ben2017.txt'

spm.SentencePieceTrainer.train(
    input=corpus_file,
    model_prefix='bengali_bpe',
    vocab_size=32000,
    model_type='bpe',
    character_coverage=0.9995,
    max_sentence_length=10000
)

print(f"Tokenizer trained and saved as {model_prefix}.model and {model_prefix}.vocab")

In [6]:
# preparing paraellel corpus
def prepare_parallel_corpus(eng_file, ben_file, output_file):
    with open(eng_file, 'r', encoding='utf-8') as eng, open(ben_file, 'r', encoding='utf-8') as ben, open(output_file, 'w', encoding='utf-8') as out:
        eng_lines = eng.readlines()
        ben_lines = ben.readlines()
        
      
        assert len(eng_lines) == len(ben_lines), "The English and Bengali files must have the same number of lines."
        
        for eng_line, ben_line in zip(eng_lines, ben_lines):
            eng_line = eng_line.strip()
            ben_line = ben_line.strip()
            if eng_line and ben_line: 
                out.write(f"{eng_line} ||| {ben_line}\n")

prepare_parallel_corpus('eng-enggnv.txt', 'ben-ben2017.txt', 'parallel_corpus.txt')

In [8]:
#tokenizing both sides of the corpus english and bengali 
